# Ngram2Vec 실행 코드 정리
* ngram2vec의 목적: 이를 바탕으로 한 감성사전 만들기!!!

## 1. 기사에 대한 라벨링
    1-1. 토큰화한 기사 파일
    1-2. 품사 정리

In [9]:
# 1. 토큰화한 기사 파일 가져오기 (test용 chunck 1만)
import pandas as pd
import ast
from tqdm import tqdm
tqdm.pandas()
test = pd.read_csv(r'/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/토큰자료/chunk_1.csv')
test['stopwords'] = test['stopwords'].apply(lambda x: ast.literal_eval(x))
# 2. 품사 정리해서 리스트 새로 만들기
ALLOWED_TAGS = {'NNG', 'VA', 'VAX', 'MAG', 'negations'}
test['stopwords'] = test['stopwords'].apply(lambda x: [word for word in x if word[1] in ALLOWED_TAGS])
test.head()
# 5000개에 13초 정도

def processChunk(chunk_num):
    chunk = pd.read_csv(f'/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/토큰자료/chunk_{chunk_num}.csv')
    chunk['stopwords'] = chunk['stopwords'].apply(lambda x: ast.literal_eval(x))


## 2. ngram 데이터 준비
    2-1. 각 text에 대한 n-gram 추출 (1~5 gram)

In [ ]:
from collections import Counter
from tqdm import tqdm

def generate_ngrams(token_list, max_n=5):
    words = [token[0] for token in token_list]
    ngram_counter = Counter()
    for n in range(1, max_n + 1):
        ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
        ngram_counter.update(ngrams)
    return ngram_counter

ngram_total = [generate_ngrams(text) for text in tqdm(test['stopwords'])]

# 5000개 2초

100%|██████████| 5000/5000 [00:02<00:00, 2044.09it/s]


## 3. N-gram 데이터를 통해 Word2Vec 데이터 학습 모델 형성
    3-1. 메모리를 잡아먹지 않도록 하나씩 문서를 처리하고 다음 문서를 불러옴
    3-2. 위의 N-gram과 합쳐서 코드 적어야 함

In [ ]:
import pandas as pd
from tqdm import tqdm
import ast
from gensim.models import Word2Vec

# n-gram 생성 함수
def generate_ngrams(token_list, max_n=5):
    words = [token[0] for token in token_list]  # 단어 리스트 추출
    ngram_list = []  # n-gram을 저장할 리스트
    for n in range(1, max_n + 1):  # 1-gram부터 max_n-gram까지
        ngrams = [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]  # n-gram을 문자열로 변환
        ngram_list.extend(ngrams)  # n-gram 리스트에 추가
    return ngram_list

# Word2Vec 모델 초기화
model = Word2Vec(vector_size=300, window=5, sg=1, negative=5, min_count=5, alpha=0.025,
                 min_alpha=0.0001, workers=4, epochs=10)

# CSV 파일 처리 함수 (각 chunk를 읽고 n-gram 리스트로 변환)
def processChunk(chunk_num):
    chunk = pd.read_csv(f'/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/토큰자료/chunk_{chunk_num}.csv')
    chunk['stopwords'] = chunk['stopwords'].apply(lambda x: ast.literal_eval(x))
    chunk['stopwords'] = chunk['stopwords'].apply(lambda x: [word for word in x if word[1] in ALLOWED_TAGS])
    return chunk

# 첫 번째 CSV 파일에서 vocab 구축
first_chunk = processChunk(1)  # 첫 번째 CSV 파일 읽기
first_ngrams = []

# 첫 번째 CSV에서 n-gram 생성
for text in first_chunk['stopwords']:
    first_ngrams.extend(generate_ngrams(text))

# 모델에 vocab을 먼저 구축 (여기서 한번만 호출)
model.build_vocab([first_ngrams], update=False)

# 첫 번째 문서를 기준으로 학습 (초기 vocab 기반)
model.train([first_ngrams], total_examples=1, epochs=model.epochs, compute_loss=True)

# 문서별로 n-gram을 생성하고 모델에 추가 (반복문 최적화)
for chunk_num in tqdm(range(2, 22)):  # 2번부터 21번까지 처리
    df = processChunk(chunk_num)  # 각 chunk 파일 읽기
    chunk_ngrams = []

    # 각 문서별로 n-gram을 생성
    for text in df['stopwords']:
        chunk_ngrams.extend(generate_ngrams(text))

    # 모델에 n-gram을 추가하고 학습 (한 번에 n-gram을 추가하고 학습)
    model.build_vocab([chunk_ngrams], update=True)
    model.train([chunk_ngrams], total_examples=1, epochs=model.epochs, compute_loss=True)

# 학습된 모델 저장
model.save("ngram2vec.model")


  5%|▌         | 1/20 [00:25<08:11, 25.86s/it]

In [60]:
# 전체 vocabulary 확인
vocab = model.wv.key_to_index  # 모든 단어의 목록
print(f"전체 vocabulary에 포함된 단어 수: {len(vocab)}")
print(f"전체 vocabulary: {list(vocab.keys())[50]}")  # 처음 10개 단어 보기
type(list(vocab.keys())[1])

전체 vocabulary에 포함된 단어 수: 73599
전체 vocabulary: 현재


str

## 4. ngram2vec 모델 테스트
    4-1. 유사도 구해지는지 확인
    4-2. 의문점이 생김; hawkish와 dovish를 어떻게 구분할 것인가? (Clustering? or 또 다른 정리법)

In [57]:
import gensim
import numpy as np

# 튜플 형태로 단어가 있을 경우 벡터 평균 구하기
def get_ngram_vector(ngram, model):
    vectors = []
    for word in ngram:
        if word in model.wv:
            vectors.append(model.wv[word])

    # 벡터가 없으면 0벡터를 반환
    if len(vectors) == 0:
        return np.zeros(model.vector_size)

    # 벡터들의 평균을 반환
    return np.mean(vectors, axis=0)

# 예시 n-gram
ngram = '하락세 같 코스닥 지수'

# Word2Vec 모델 로드
# 모델은 이미 로드되어 있다고 가정
model = gensim.models.Word2Vec.load('ngram2vec.model')

# n-gram의 벡터 확인
vector = get_ngram_vector(ngram, model)
print(f"'{ngram}'의 벡터: {vector}")


'하락세 같 코스닥 지수'의 벡터: [ 4.74831555e-03 -4.44567949e-03  1.03894342e-02  1.06598940e-02
  7.67382002e-03 -4.07999046e-02 -1.36427041e-02  5.63830361e-02
  3.48775042e-03  5.34613244e-03 -1.00443903e-02  5.47024654e-03
  1.64841153e-02  1.31024970e-02 -2.10052636e-02 -2.01698532e-03
  8.64014309e-03 -1.81387160e-02  1.09828413e-02 -1.34580331e-02
 -1.89270843e-02 -7.98604079e-03  2.41787955e-02  1.72139741e-02
  3.43264639e-02 -2.46964861e-03 -3.74869891e-02  1.04097435e-02
 -1.28866937e-02 -1.91779733e-02 -1.21169481e-02  1.54829873e-02
  1.13289803e-02  1.99607224e-03 -1.41040552e-02  1.38424207e-02
  1.16062602e-02 -1.46797691e-02  1.55150322e-02 -7.38779223e-03
 -4.17387933e-02  5.92664233e-04  1.32148238e-02 -1.83257256e-02
  2.46645175e-02  1.47758294e-02 -5.67368697e-03  2.31476612e-02
  2.19245767e-03  2.31985394e-02  1.26400087e-02 -1.78823117e-02
  5.21510188e-03 -1.19959284e-02 -9.46885534e-03 -2.86781765e-03
  1.70562845e-02  1.40523566e-02 -1.82484258e-02  3.59073724e-03
 -5.4

In [59]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gensim

model = gensim.models.Word2Vec.load('ngram2vec.model')
# n-gram을 벡터로 변환하는 함수 (각 단어의 벡터 평균 계산)
def get_ngram_vector(ngram, model):
    # ngram이 튜플일 경우, 각 단어 벡터를 가져와서 평균을 구함
    vectors = [model.wv[word] for word in ngram if word in model.wv]

    if len(vectors) == 0:
        return np.zeros(model.vector_size)  # 벡터가 없으면 0으로 대체

    # 벡터들의 평균을 계산
    return np.mean(vectors, axis=0)

# 두 n-gram 간의 유사도 계산
def calculate_similarity(ngram1, ngram2, model):
    vec1 = get_ngram_vector(ngram1, model)
    vec2 = get_ngram_vector(ngram2, model)

    # 벡터가 NaN인 경우를 처리
    if np.isnan(vec1).any() or np.isnan(vec2).any():
        return 0  # NaN이 있을 경우 유사도를 0으로 처리

    # 벡터를 2D 배열로 reshape하여 cosine similarity 계산
    similarity = cosine_similarity([vec1], [vec2])[0][0]
    return similarity

# 예시 n-gram 튜플
ngram1 = '금리 하락' # list(vocab.keys())[500]
ngram2 = '미국'# list(vocab.keys())[]

# Word2Vec 모델 로드 (이미 학습된 모델을 사용)
# 예시: 모델을 로드할 때 사용하는 코드 (이미 학습된 모델이 있어야 함)

# 두 n-gram 튜플의 유사도 계산
similarity = calculate_similarity(ngram1, ngram2, model)
print(f"'{ngram1}'와 '{ngram2}'의 유사도: {similarity}")


'금리 하락'와 '미국'의 유사도: 0.923205554485321


In [ ]:
# hawkish와 dovish 사전 구해서 만들어봐야할 듯 -> 어떻게?

## 5. 최적화
    5-1. 우리가 처리해야 할 데이터는 text만 총 25만 개 정도!
    5-2. 어떻게 하면 멈추거나 메모리 문제 없이 제대로 해결할 수 있을까 -> 지금 모델로는 빠르게 돌리지는 못해도 메모리 충돌이나 과다는 없었음
